In [1]:
import pandas as pd, numpy as np
import vivarium_output_loader as vol
import lsff_output_processing as lop

!whoami
!date

ndbs
Fri May 22 13:10:03 PDT 2020


In [2]:
%load_ext autoreload
%autoreload 2

## 1. Define directories and load count space data

In [5]:
base_directory = '/share/costeffectiveness/results/vivarium_conic_lsff/00_bugfix'
output_directory = base_directory + '/v6.0_prelim_maternal_iron_lri_bp/india_5_20'

In [6]:
data = vol.load_transformed_count_data(f'{output_directory}/count_data')
data.keys()

dict_keys(['gestational_age', 'transition_count', 'deaths', 'state_person_time', 'births_with_ntd', 'population', 'person_time', 'ylls', 'ylds', 'births', 'birth_weight'])

## 2. Look at deaths table; check scenarios and stratification

*  No iron fortification
*  Stratification not updated to new group definitions

In [7]:
data['deaths'].head()

,year,age_group,sex,cause,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,female,diarrheal_diseases,covered,covered,death,29,baseline,0.0
1,2020,1_to_4,female,diarrheal_diseases,covered,covered,death,29,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,female,diarrheal_diseases,covered,covered,death,29,vitamin_a_fortification_scale_up,0.0
3,2020,1_to_4,female,diarrheal_diseases,covered,covered,death,232,baseline,0.0
4,2020,1_to_4,female,diarrheal_diseases,covered,covered,death,232,folic_acid_fortification_scale_up,0.0


In [8]:
data['deaths'].scenario.unique()

array(['baseline', 'folic_acid_fortification_scale_up',
       'vitamin_a_fortification_scale_up'], dtype=object)

In [9]:
data['deaths'].folic_acid_fortification_group.unique()

array(['covered', 'uncovered', 'unknown'], dtype=object)

In [10]:
data['deaths'].vitamin_a_fortification_group.unique()

array(['covered', 'effectively_covered', 'uncovered'], dtype=object)

## 3. Look at births and births_with_ntds tables

In [12]:
data['births'].head()

,year,sex,folic_acid_fortification_group,measure,input_draw,scenario,value
0,2020,female,covered,live_births,29,baseline,327.0
1,2020,female,covered,live_births,29,baseline,5.0
2,2020,female,covered,live_births,29,baseline,80.0
3,2020,female,covered,live_births,29,folic_acid_fortification_scale_up,233.0
4,2020,female,covered,live_births,29,folic_acid_fortification_scale_up,4.0


In [13]:
data['births_with_ntd'].head()

,year,sex,folic_acid_fortification_group,measure,input_draw,scenario,value
0,2020,female,covered,live_births_with_ntds,29,baseline,0.0
1,2020,female,covered,live_births_with_ntds,29,baseline,0.0
2,2020,female,covered,live_births_with_ntds,29,baseline,0.0
3,2020,female,covered,live_births_with_ntds,29,folic_acid_fortification_scale_up,0.0
4,2020,female,covered,live_births_with_ntds,29,folic_acid_fortification_scale_up,0.0


In [14]:
data['births'].scenario.unique()

array(['baseline', 'folic_acid_fortification_scale_up',
       'vitamin_a_fortification_scale_up'], dtype=object)

## 4. Check state person time and person time tables

In [15]:
data['state_person_time'].head()

,year,age_group,sex,cause,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,29,baseline,0.0
1,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,29,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,29,vitamin_a_fortification_scale_up,0.0
3,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,232,baseline,0.0
4,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,232,folic_acid_fortification_scale_up,0.0


In [16]:
data['state_person_time'].cause.unique()

array(['diarrheal_diseases', 'lower_respiratory_infections', 'measles',
       'neural_tube_defects', 'susceptible_to_diarrheal_diseases',
       'susceptible_to_lower_respiratory_infections',
       'susceptible_to_measles', 'susceptible_to_neural_tube_defects',
       'susceptible_to_vitamin_a_deficiency', 'vitamin_a_deficiency'],
      dtype=object)

In [17]:
data['person_time'].head()

,year,age_group,sex,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,female,covered,covered,person_time,29,baseline,0.0
1,2020,1_to_4,female,covered,covered,person_time,29,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,female,covered,covered,person_time,29,vitamin_a_fortification_scale_up,0.0
3,2020,1_to_4,female,covered,covered,person_time,232,baseline,0.0
4,2020,1_to_4,female,covered,covered,person_time,232,folic_acid_fortification_scale_up,0.0


## 5. Compute prevalences by age group

In [18]:
numerator_strata = ['year', 'age_group']
denominator_strata = ['year', 'age_group']
multiplier = 100 # Get prevalences as percents
broadcast_cols = ['cause'] # Broadcast over cause column in state person time

prevalence = lop.rate_or_ratio(data['state_person_time'],
                               data['person_time'],
                               numerator_strata,
                               denominator_strata,
                               multiplier,
                               broadcast_cols,
                              )
prevalence.head()

,year,age_group,input_draw,scenario,cause,value
0,2020,1_to_4,29,baseline,diarrheal_diseases,1.257451
1,2020,1_to_4,29,baseline,lower_respiratory_infections,0.281531
2,2020,1_to_4,29,baseline,measles,0.115051
3,2020,1_to_4,29,baseline,neural_tube_defects,0.050326
4,2020,1_to_4,29,baseline,susceptible_to_diarrheal_diseases,98.199246


## 6. Test some pandas functions

In [19]:
prevalence.stack().head(10)

0  year                        2020
   age_group                 1_to_4
   input_draw                    29
   scenario                baseline
   cause         diarrheal_diseases
   value                    1.25745
1  year                        2020
   age_group                 1_to_4
   input_draw                    29
   scenario                baseline
dtype: object